In [1]:
!pip install praw
import praw
import pandas as pd
import time

In [6]:
import requests
import csv
import time

def fetch_all_threads(subreddit, max_pages=10, delay=2):
    headers = {'User-Agent': 'RedditScraper/1.0'}
    base_url = f'https://www.reddit.com/r/formula1/.json'
    after = None
    all_posts = []

    for _ in range(max_pages):
        url = base_url + (f'?after={after}' if after else '')
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break

        data = response.json()
        posts = data['data']['children']
        all_posts.extend([post['data'] for post in posts])

        after = data['data'].get('after')
        if not after:
            break

        time.sleep(delay)

    return all_posts

def write_to_csv(posts, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['title', 'selftext'])
        writer.writeheader()

        for post in posts:
            writer.writerow({
                'title': post.get('title', '').replace('\n', ' ').strip(),
                'selftext': post.get('selftext', '').replace('\n', ' ').strip()
            })

# Usage
subreddit = 'formula1'
threads = fetch_all_threads(subreddit, max_pages=5)
csv_filename = f'{subreddit}_threads.csv'

write_to_csv(threads, csv_filename)
print(f"Wrote {len(threads)} threads to {csv_filename}")

Wrote 127 threads to formula1_threads.csv
